# Bug report

In [1]:
from coronado import CoronadoUnprocessableObjectError 
from coronado.address import Address
from coronado.reward import Reward
from coronado.reward import SERVICE_PATH

import json
import uuid

import coronado.auth as auth

conf = auth.loadConfig()
auth = auth.Auth(conf['tokenURL'], clientID = conf['clientID'], clientSecret = conf['secret'], scope = auth.Scope.CONTENT_PROVIDERS)

Reward.initialize(conf['serviceURL'], SERVICE_PATH, auth)

## Formatting stuff

In [2]:
import pandas as pd  # We'll use this for pretty printing in this document

def prettyPrintListOf(tripleClass, indexName = 'objID', **listArgs):
    panel = pd.DataFrame([ tripleObject.__dict__ for tripleObject in tripleClass.list(**listArgs) ])
    panel.index = panel[indexName]
    del panel[indexName]
    return panel

## Known values for various initializers

In [3]:
KNOWN_ACCT_EXT_ID = 'pnc-card-69-3149b4780d6f4c2fa21fb45d2637efbb'
KNOWN_ACCT_ID = '1270'
KNOWN_CARD_PROG_EXT_ID = 'prog-66'
KNOWN_PUB_EXTERNAL_ID = '0d7c608a3df5'
KNOWN_CARD_ACCT_EXT_ID = 'pnc-card-69-0b10aa350dec4201be3107f7aca060f2'
KNOWN_MCC = '0780'

# Bug!

Based on the workflow semantics and error message, the `/rewards.approve` and `/rewards.deny` services should return a 404 instead of a 422 when receiving invalid `transactionID` or `offerID`.  If any of the required attributes is **missing** then 422 is the correct response.

In [4]:
result = Reward.approve('bogus-transaction', '42')
print('approval result = %s' % result)
print('service response: %s' % Reward.responseDDT)

approval result = False
service response: {'detail': 'No reward found for transaction_id "bogus-transaction" and offer_id "42".'}


This lead to kludgy code to handle the missing `notes` attribute for the `/rewards.deny` handler -- this code is unnecesary if the service implements the correct 404 / 422 semantics:

```python
    # TODO: This is a legit 422; the others aren't.
    if None == notes and 'deny' == action:
        raise CoronadoMalformedObjectError('notes attribute missing or set to None')

    if '' == notes and 'deny' == action:
        raise CoronadoMalformedObjectError(
            'notes attribute must have some text; empty strings disallowed')

    endpoint = '/'.join([ klass._serviceURL, 'partner/rewards.%s' % action ])
    response = requests.request('POST', endpoint, headers = klass.headers, json = spec)
```